In [1]:
import Ipynb_importer
from a_basic_quant import *
from b_model import *
from c_train_and_test import *
from d_post_training_quantize import *

importing Jupyter notebook from a_basic_quant.ipynb
importing Jupyter notebook from b_model.ipynb
importing Jupyter notebook from c_train_and_test.ipynb
importing Jupyter notebook from d_post_training_quantize.ipynb


In [1]:
import torch

In [6]:
a = torch.randn((64,1,28,28))
a.shape

torch.Size([64, 1, 28, 28])

In [7]:
conv = torch.nn.Conv2d(1, 40, 3, 1)
res = conv(a)

In [8]:
res.shape

torch.Size([64, 40, 26, 26])

In [9]:
res = res.permute(1,0,2,3)

In [10]:
res.shape

torch.Size([40, 64, 26, 26])

In [ ]:
max =[float(res[i,:,:,:].max() )for i in range(len(res))] 
max


In [ ]:
min = [float(res[i,:,:,:].min() )for i in range(len(res))] 
min

In [31]:
len(res)

40